# 1、ChatMessageHistory的使用


场景1：记忆存储

In [1]:
from langchain.memory import ChatMessageHistory

# 1、ChatMessageHistory的实例化
history = ChatMessageHistory()

# 2、添加相关的消息进行存储
history.add_user_message('你好')
history.add_ai_message('很高兴认识你')

# 3、打印存储的消息
print(history.messages)

[HumanMessage(content='你好', additional_kwargs={}, response_metadata={}), AIMessage(content='很高兴认识你', additional_kwargs={}, response_metadata={})]


场景2：对接LLM

In [2]:
# 1、获取大模型
import os
import dotenv
from langchain_openai import ChatOpenAI

# 加载环境变量
dotenv.load_dotenv()

os.environ['OPENAI_BASE_URL'] = os.getenv('OPENAI_BASE_URL')
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

# 创建大模型实例
llm = ChatOpenAI(model='gpt-4o-mini')



In [3]:
from langchain.memory import ChatMessageHistory

# 1、ChatMessageHistory的实例化
history = ChatMessageHistory()

# 2、添加相关的消息进行存储
history.add_user_message('你好')
history.add_ai_message('很高兴认识你')
history.add_user_message('帮我计算 1 + 2 * 3 = ?')

response = llm.invoke(history.messages)
print(response.content)

根据数学运算的优先级，首先进行乘法运算，然后再进行加法运算。因此：

1 + 2 * 3 = 1 + 6 = 7

所以，结果是 7。


# 2、ConversationBufferMemory的使用


举例1：以字符串的形式返回存储的消息

In [6]:
from langchain.memory import ConversationBufferMemory

# 1、ConversationBufferMemory的实例化
memory = ConversationBufferMemory()

# 2、存储相关的消息
# input对应的就是用户的消息, output对应的就是AI的消息
memory.save_context(inputs={'human': '你好, 我叫小明'}, outputs={'ai': '很高兴认识你'})
memory.save_context(inputs={'input': '帮我回答一下 1 + 2 * 3 = ?'}, outputs={'output': '7'})

# 3、获取存储的消息
print(memory.load_memory_variables({}))

# 返回的字典结构的key叫history

{'history': 'Human: 你好, 我叫小明\nAI: 很高兴认识你\nHuman: 帮我回答一下 1 + 2 * 3 = ?\nAI: 7'}


举例2：以消息列表的形式返回存储的消息

In [8]:
from langchain.memory import ConversationBufferMemory

# 1、ConversationBufferMemory的实例化
memory = ConversationBufferMemory(return_messages=True)

# 2、存储相关的消息
# input对应的就是用户的消息, output对应的就是AI的消息
memory.save_context(inputs={'human': '你好, 我叫小明'}, outputs={'ai': '很高兴认识你'})
memory.save_context(inputs={'input': '帮我回答一下 1 + 2 * 3 = ?'}, outputs={'output': '7'})

# 3、获取存储的消息
# 返回消息列表的方式1
print(memory.load_memory_variables({}))

print()

# 返回消息列表的方式2
print(memory.chat_memory.messages)

# 返回的字典结构的key叫history

{'history': [HumanMessage(content='你好, 我叫小明', additional_kwargs={}, response_metadata={}), AIMessage(content='很高兴认识你', additional_kwargs={}, response_metadata={}), HumanMessage(content='帮我回答一下 1 + 2 * 3 = ?', additional_kwargs={}, response_metadata={}), AIMessage(content='7', additional_kwargs={}, response_metadata={})]}

[HumanMessage(content='你好, 我叫小明', additional_kwargs={}, response_metadata={}), AIMessage(content='很高兴认识你', additional_kwargs={}, response_metadata={}), HumanMessage(content='帮我回答一下 1 + 2 * 3 = ?', additional_kwargs={}, response_metadata={}), AIMessage(content='7', additional_kwargs={}, response_metadata={})]


举例3：结合提示词模板（PromptTemplate）、大模型的使用

In [11]:
from langchain.chains.llm import LLMChain
from langchain_core.prompts import PromptTemplate
# 1、创建大模型
import os
import dotenv
from langchain_openai import ChatOpenAI

# 加载环境变量
dotenv.load_dotenv()

# 给系统环境变量赋值
os.environ['OPENAI_BASE_URL'] = os.getenv('OPENAI_BASE_URL')
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

# 获取大模型
llm = ChatOpenAI(model='gpt-4o-mini')

# 2、创建提示词模板
prompt_template = PromptTemplate.from_template(
    template="""你可以与人类对话。
    
            当前对话历史: {history}
            
            人类问题: {question}
            
            回复:
            """
)

# 3、提供Memory的实例
memory = ConversationBufferMemory()

# 4、提供Chain
# 此处提示词模板中存在两个变量, 但是只传入了一个question变量却不报错, 是因为memory会返回一个以key为history的值, 会间接传入到提示词中
chain = LLMChain(llm=llm, prompt=prompt_template, memory=memory)

response = chain.invoke({'question': '你好, 我的名字叫小明'})
print(response)

{'question': '你好, 我的名字叫小明', 'history': '', 'text': '你好，小明！很高兴认识你！有什么我可以帮助你的吗？'}


In [12]:
response = chain.invoke('我叫什么名字呢？')
print(response)

{'question': '我叫什么名字呢？', 'history': 'Human: 你好, 我的名字叫小明\nAI: 你好，小明！很高兴认识你！有什么我可以帮助你的吗？', 'text': '你叫小明！很高兴再次见到你！你有什么想聊的呢？'}


举例4：基于举例3, 显示得设置memory的key值

In [13]:
from langchain.chains.llm import LLMChain
from langchain_core.prompts import PromptTemplate
# 1、创建大模型
import os
import dotenv
from langchain_openai import ChatOpenAI

# 加载环境变量
dotenv.load_dotenv()

# 给系统环境变量赋值
os.environ['OPENAI_BASE_URL'] = os.getenv('OPENAI_BASE_URL')
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

# 获取大模型
llm = ChatOpenAI(model='gpt-4o-mini')

# 2、创建提示词模板
prompt_template = PromptTemplate.from_template(
    template="""你可以与人类对话。
    
            当前对话历史: {chat_history}
            
            人类问题: {question}
            
            回复:
            """
)

# 3、提供Memory的实例
# 此处需要将memory_key自定义的值与上面提示词模板中的变量统一起来即可
memory = ConversationBufferMemory(memory_key='chat_history')

# 4、提供Chain
chain = LLMChain(llm=llm, prompt=prompt_template, memory=memory)

response = chain.invoke({'question': '你好, 我的名字叫小明'})
print(response)

{'question': '你好, 我的名字叫小明', 'chat_history': '', 'text': '你好，小明！很高兴认识你。有什么我可以帮助你的吗？'}


In [14]:
response = chain.invoke('我叫什么名字呢？')
print(response)

{'question': '我叫什么名字呢？', 'chat_history': 'Human: 你好, 我的名字叫小明\nAI: 你好，小明！很高兴认识你。有什么我可以帮助你的吗？', 'text': '你叫小明。有什么你想聊的吗？'}


5、结合提示词模板（ChatPromptTemplate）, 大模型使用

In [15]:
# 1.导入相关包
from langchain_core.messages import SystemMessage
from langchain.chains.llm import LLMChain
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import MessagesPlaceholder, ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_openai import ChatOpenAI

# 2.创建LLM
llm = ChatOpenAI(model_name='gpt-4o-mini')

# 3.创建Prompt
prompt = ChatPromptTemplate.from_messages([
    ("system", "你是一个与人类对话的机器人。"),
    MessagesPlaceholder(variable_name='history'),
    ("human", "问题：{question}")
])

# 4.创建Memory
memory = ConversationBufferMemory(return_messages=True)

# 5.创建LLMChain
llm_chain = LLMChain(prompt=prompt, llm=llm, memory=memory)

# 6.调用LLMChain
res1 = llm_chain.invoke({"question": "中国首都在哪里？"})
print(res1, end="\n\n")

{'question': '中国首都在哪里？', 'history': [HumanMessage(content='中国首都在哪里？', additional_kwargs={}, response_metadata={}), AIMessage(content='中国的首都是北京。', additional_kwargs={}, response_metadata={})], 'text': '中国的首都是北京。'}


In [16]:
res2 = llm_chain.invoke({"question": "我刚刚问了什么"})
print(res2, end="\n\n")

{'question': '我刚刚问了什么', 'history': [HumanMessage(content='中国首都在哪里？', additional_kwargs={}, response_metadata={}), AIMessage(content='中国的首都是北京。', additional_kwargs={}, response_metadata={}), HumanMessage(content='我刚刚问了什么', additional_kwargs={}, response_metadata={}), AIMessage(content='你问了“中国首都在哪里？”这个问题。', additional_kwargs={}, response_metadata={})], 'text': '你问了“中国首都在哪里？”这个问题。'}
